In [1]:
import pydicom
import csv
import re
import pandas as pd

In [2]:
import os

Aqui começa por uma tentativa de tentar extrair as clinical features dos pacientes da margarida que não existiam e que depois foi possível, através dos ficheiros de plano, porque não tinha a informação noutro lado. A informação extra que consegui nomeadamente data de tratamento, follow-up, decisão, tumor primário e idade que na verdade foi extraída através da data de nascimento (fiz na altura um código para tal, porque nos ficheiros só tem data de nascimento) criei um excel no meu computador para tal e depois submeti aqui e transformei-o para csv. No entanto, fi-lo no notebook  Distribution Clinical features que está na pasta notebooks > Extra do meu container, porque na altura ainda não os pacientes todos e tinha que entregar um artigo e não queria "estragar" os notebooks steps que tenho aqui. Passei, contudo, os blocos de código que utilizei lá para transformar em csv para aqui. A distribuição de clinical features continua lá para depois fazer análise de características dos paciente.

### Clinical Features from the RT files

In [3]:
image = "/notebooks/container_margarida/RT_Dataset/04_Anonymized_Patients/Patient_17/P17_CT_1_TRATAMENTO/RTPLAN.dcm"
df = pydicom.dcmread(image, force=True)
df

(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0012) Instance Creation Date              DA: '20240313'
(0008, 0013) Instance Creation Time              TM: '135236'
(0008, 0016) SOP Class UID                       UI: RT Plan Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.840.113854.2829848148.719929522.3043916264.2471068602
(0008, 0020) Study Date                          DA: '20201023'
(0008, 0030) Study Time                          TM: '091741'
(0008, 0050) Accession Number                    SH: ''
(0008, 0060) Modality                            CS: 'RTPLAN'
(0008, 0070) Manufacturer                        LO: 'Elekta'
(0008, 0090) Referring Physician's Name          PN: ''
(0008, 1030) Study Description                   LO: 'CRANIO/GK'
(0008, 1070) Operators' Name                     PN: ''
(0008, 1080) Admitting Diagnoses Description     LO: 'Metastases - multiple'
(0008, 1084)  Admitting Diagnoses Code Sequence   1 item(s) ---- 
  

In [4]:
image = "/notebooks/container_margarida/RT_Dataset/07_ClinicalFeatures/RTPLAN_P2.dcm"
df = pydicom.dcmread(image, force=True)

name = (0x0010,0x0010) 
id = (0x0010,0x0020)

patient_name = df[name].value
patient_name = str(patient_name)
print(patient_name)
df.PatientID = patient_name.replace("tentativa 2", "").replace(" ","")


df.save_as(image)

P2-25 tentativa 2


In [16]:
#Extracting clinical features  
clinical_data={}
def extracting_clinical_features(df, output_folder='/notebooks/disk2/CSV_Dataset'):
    id = (0x0010, 0x0020)
    sex = (0x0010, 0x0040)
    #age = (0x0010, 0x1010)
    dose_reference_seq_tag = (0x300A, 0x0010)  # Dose Reference Sequence
    dose_reference_type_tag = (0x300A, 0x0020)  # Dose Reference Type
    name = (0x0010,0x0010) 
    machine = (0x0008,0x1090) # Manufacturer's Model Name
    # Extract data from the file
    data = df[id].value
    print(data)

    number = 0
    patient_sex = df[sex].value
    patient_id = df[id].value
    machine_treatment = df[machine].value

    if patient_id == '':
        patient_id = df[name].value
        
    patient_id = str(patient_id)
    patient_id = patient_id.replace("P", "Patient_")
    patient_id = patient_id.replace("__","_")

    print("Extracting patient's data...")
    print(f"Patient ID: {patient_id}")
    print(f"Patient Sex: {patient_sex}")
    #print(f"Patient Age: {patient_age}")

    if "GammaPlan" ==  machine_treatment:
        machine_treatment = "Gamma Knife"
    elif "Monaco" == machine_treatment:
        machine_treatment = "LINAC"
    else:
        mac = str(input("Insert please the type of machine used - LINAC or Gamma Knife"))
        if mac == "LINAC" or "Gamma Knife":
            machine_treatment = mac
        else:
            print("Error")

    if dose_reference_seq_tag in df:
        dose_reference_sequence = df[dose_reference_seq_tag]
        for item in dose_reference_sequence:
            if dose_reference_type_tag in item:
                dose_reference_type = item[dose_reference_type_tag].value
                if dose_reference_type == 'TARGET':
                    number = number + 1
                    #print(number)
                else:
                    print("No metastases")
            else:
                print("Not found!!")

    #print(f"Cleaned Patient ID: {patient_id}")
    #print(f"Cleaned Patient Sex: {patient_sex}")
    #print(f"Cleaned Patient Age: {age_numerical}")

    clinical_features = {
        'Patient ID': patient_id,
        'Sex': patient_sex,
        'Number of metastases': number,
        'Machine' : machine_treatment
    }

    print("Data extraction completed with success", clinical_features)
    
    os.makedirs(output_folder, exist_ok=True)
  
    csv_file = os.path.join(output_folder, 'Clinical_Features.csv')
    header = ['Patient ID', 'Sex', 'Number of metastases', 'Machine']
        
    with open(csv_file, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=header)
        if file.tell() == 0:
            writer.writeheader()
        writer.writerow(clinical_features)
    return clinical_features

In [ ]:
folder_path = '/notebooks/container_margarida/RT_Dataset/07_ClinicalFeatures'
folder_path_2 = 

if os.path.exists(folder_path):
    print("Data found!")
else:
    print("Data not found. Please, confirm the directory.")

all_files = os.listdir(folder_path)
all_files_2 = os.listdir(folder_path_2)

for file in all_files:
    full_path = os.path.join(folder_path, file)
    if os.path.isfile(full_path) and file.endswith(".dcm"):
        try:
            df = pydicom.dcmread(full_path, force=True)
            extracting_clinical_features(df, output_folder='/notebooks/disk2/CSV_Dataset')
        except Exception as e:
            print("Not found!")

for file in all_files_2:
    full_path_2 = os.path.join(folder_path_2,file)
    if os.path.isfile(full_path_2) and file.endswith(".dcm"):
        try:
            df = pydicom.dcmread(full_path_2, force=True)
            extracting_clinical_features(df, output_folder='/notebooks/disk2/CSV_Dataset')
        except Exception as e:
            print("Not found!")
        

In [17]:
folder_path = "/notebooks/container_margarida/RT_Dataset/04_Anonymized_Patients"
folder_path_2 = "/notebooks/disk2/RT_Dataset_2/01_Anonymized_Patients"

if os.path.exists(folder_path):
    print("Data found!")
else:
    print("Data not found. Please, confirm the directory.")

all_files = os.listdir(folder_path)
all_files_2 = os.listdir(folder_path_2)

for root, dirs, files in os.walk(folder_path):
    for file in files:
        if "CT_1_TRATAMENTO" in root and ("RTPLAN" in file or "DCMRT_" in file):
            full_path = os.path.join(root, file)
            df = pydicom.dcmread(full_path, force = True)
            extracting_clinical_features(df, output_folder='/notebooks/disk2/CSV_Dataset')
            
for root, dirs, files in os.walk(folder_path_2):
    for file in files:
        if "CT_1_TRATAMENTO" in root and ("RTPLAN" in file or "DCMRT_" in file):
            full_path_2 = os.path.join(root, file)
            df = pydicom.dcmread(full_path_2, force = True)
            extracting_clinical_features(df, output_folder='/notebooks/disk2/CSV_Dataset')
    

Data found!

Extracting patient's data...
Patient ID: Patient_17
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_17', 'Sex': 'F', 'Number of metastases': 7, 'Machine': 'Gamma Knife'}

Extracting patient's data...
Patient ID: Patient_50
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_50', 'Sex': 'F', 'Number of metastases': 0, 'Machine': 'LINAC'}

Extracting patient's data...
Patient ID: Patient_50
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_50', 'Sex': 'F', 'Number of metastases': 2, 'Machine': 'LINAC'}

Extracting patient's data...
Patient ID: Patient_50
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_50', 'Sex': 'F', 'Number of metastases': 2, 'Machine': 'LINAC'}

Extracting patient's data...
Patient ID: Patient_50
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_50', 'Sex': 'F', 'Number of metastases': 2, 'Machine': 'LINAC'}

Extrac

Insert please the type of machine used - LINAC or Gamma Knife LINAC


Data extraction completed with success {'Patient ID': 'Patient_2-25', 'Sex': 'F', 'Number of metastases': 4, 'Machine': 'LINAC'}

Extracting patient's data...
Patient ID: Patient_75
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_75', 'Sex': 'F', 'Number of metastases': 2, 'Machine': 'Gamma Knife'}

Extracting patient's data...
Patient ID: Patient_84
Patient Sex: M
Data extraction completed with success {'Patient ID': 'Patient_84', 'Sex': 'M', 'Number of metastases': 0, 'Machine': 'LINAC'}

Extracting patient's data...
Patient ID: Patient_84
Patient Sex: M
Data extraction completed with success {'Patient ID': 'Patient_84', 'Sex': 'M', 'Number of metastases': 2, 'Machine': 'LINAC'}

Extracting patient's data...
Patient ID: Patient_88
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_88', 'Sex': 'F', 'Number of metastases': 0, 'Machine': 'LINAC'}

Extracting patient's data...
Patient ID: Patient_88
Patient Sex: F
Data extraction c

Insert please the type of machine used - LINAC or Gamma Knife LINAC


Data extraction completed with success {'Patient ID': 'Patient_1-25', 'Sex': 'F', 'Number of metastases': 10, 'Machine': 'LINAC'}

Extracting patient's data...
Patient ID: Patient_68
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_68', 'Sex': 'F', 'Number of metastases': 0, 'Machine': 'LINAC'}

Extracting patient's data...
Patient ID: Patient_68
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_68', 'Sex': 'F', 'Number of metastases': 2, 'Machine': 'LINAC'}

Extracting patient's data...
Patient ID: Patient_68
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_68', 'Sex': 'F', 'Number of metastases': 2, 'Machine': 'LINAC'}

Extracting patient's data...
Patient ID: Patient_63
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_63', 'Sex': 'F', 'Number of metastases': 0, 'Machine': 'LINAC'}

Extracting patient's data...
Patient ID: Patient_63
Patient Sex: F
Data extraction comple

Insert please the type of machine used - LINAC or Gamma Knife LINAC


Data extraction completed with success {'Patient ID': 'Patient_3-25', 'Sex': 'F', 'Number of metastases': 10, 'Machine': 'LINAC'}

Extracting patient's data...
Patient ID: Patient_81
Patient Sex: M
Data extraction completed with success {'Patient ID': 'Patient_81', 'Sex': 'M', 'Number of metastases': 0, 'Machine': 'LINAC'}

Extracting patient's data...
Patient ID: Patient_81
Patient Sex: M
Data extraction completed with success {'Patient ID': 'Patient_81', 'Sex': 'M', 'Number of metastases': 2, 'Machine': 'LINAC'}

Extracting patient's data...
Patient ID: Patient_83
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_83', 'Sex': 'F', 'Number of metastases': 0, 'Machine': 'LINAC'}

Extracting patient's data...
Patient ID: Patient_83
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_83', 'Sex': 'F', 'Number of metastases': 2, 'Machine': 'LINAC'}

Extracting patient's data...
Patient ID: Patient_91
Patient Sex: F
Data extraction comple

In [18]:
import pydicom
import csv
import os

clinical_data = {} 
output_folder='/notebooks/disk2/CSV_Dataset'
os.makedirs(output_folder, exist_ok=True)
csv_file = os.path.join(output_folder, 'Clinical_Features.csv')
header = ['Patient ID', 'Sex', 'Number of metastases', 'Machine']

if os.path.exists(csv_file):
    import pandas as pd
    df = pd.read_csv(csv_file)
    
    # Add info from the CSV to the dict clinical_data to accumulate metastases
    for _, row in df.iterrows():
        patient_id = row['Patient ID']
        if patient_id not in clinical_data:
            clinical_data[patient_id] = {
                'Sex': row['Sex'],
                'Number of metastases': row['Number of metastases'],
                'Machine': row['Machine']
            }

with open(csv_file, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=header)
    writer.writeheader()

    m = 0
    for root, dirs, files in os.walk('/notebooks/container_margarida/RT_Dataset/04_Anonymized_Patients'):  
        for file_name in files:
            if "CT_1_TRATAMENTO" in root and ("DCMRT_Plan" in file_name or "RTPLAN" in file_name):
                full_path = os.path.join(root, file_name)
                df = pydicom.dcmread(full_path, force=True)
                extracted_data = extracting_clinical_features(df, output_folder='/notebooks/disk2/CSV_Dataset') 
                patient_id = extracted_data['Patient ID']

                print(f"Processing patient: {patient_id}")

                # Verifying if Patient ID is already in the clinical_data dict
                if patient_id in clinical_data:
                    print(f"Patient {patient_id} already exists.")
                    prev_met = clinical_data[patient_id]["Number of metastases"]
                    new_met = extracted_data["Number of metastases"]

                    clinical_data[patient_id]["Number of metastases"] = prev_met + new_met
                else:
                    # Adicionar um novo paciente ao dicionário
                    clinical_data[patient_id] = extracted_data
                    print(f"This patient is new!")


  #CSV without dups
    for patient_id, data in clinical_data.items():
        writer.writerow({
            'Patient ID': patient_id,
            'Sex': data['Sex'],
            'Number of metastases': data['Number of metastases'],
            'Machine': data['Machine']
        })

    print(f"File '{csv_file}' updated successfully!")




Extracting patient's data...
Patient ID: Patient_17
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_17', 'Sex': 'F', 'Number of metastases': 7, 'Machine': 'Gamma Knife'}
Processing patient: Patient_17
Patient Patient_17 already exists.

Extracting patient's data...
Patient ID: Patient_50
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_50', 'Sex': 'F', 'Number of metastases': 2, 'Machine': 'LINAC'}
Processing patient: Patient_50
Patient Patient_50 already exists.

Extracting patient's data...
Patient ID: Patient_50
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_50', 'Sex': 'F', 'Number of metastases': 2, 'Machine': 'LINAC'}
Processing patient: Patient_50
Patient Patient_50 already exists.

Extracting patient's data...
Patient ID: Patient_50
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_50', 'Sex': 'F', 'Number of metastases': 2, 'Machine': 'LINAC'}
Processing patie

In [19]:
import pydicom
import csv
import os

clinical_data = {} 
output_folder='/notebooks/disk2/CSV_Dataset'
os.makedirs(output_folder, exist_ok=True)
csv_file = os.path.join(output_folder, 'Clinical_Features.csv')
header = ['Patient ID', 'Sex', 'Number of metastases', 'Machine']

if os.path.exists(csv_file):
    import pandas as pd
    df = pd.read_csv(csv_file)
    
    # Add info from the CSV to the dict clinical_data to accumulate metastases
    for _, row in df.iterrows():
        patient_id = row['Patient ID']
        if patient_id not in clinical_data:
            clinical_data[patient_id] = {
                'Sex': row['Sex'],
                'Number of metastases': row['Number of metastases'],
                'Machine': row['Machine']
            }

with open(csv_file, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=header)
    writer.writeheader()

    m = 0
    for root, dirs, files in os.walk('/notebooks/disk2/RT_Dataset_2/01_Anonymized_Patients'):  
        for file_name in files:
            if "CT_1_TRATAMENTO" in root and ("DCMRT_Plan" in file_name or "RTPLAN" in file_name):
                full_path = os.path.join(root, file_name)
                df = pydicom.dcmread(full_path, force=True)
                extracted_data = extracting_clinical_features(df, output_folder='/notebooks/disk2/CSV_Dataset') 
                patient_id = extracted_data['Patient ID']

                print(f"Processing patient: {patient_id}")

                # Verifying if Patient ID is already in the clinical_data dict
                if patient_id in clinical_data:
                    print(f"Patient {patient_id} already exists.")
                    prev_met = clinical_data[patient_id]["Number of metastases"]
                    new_met = extracted_data["Number of metastases"]

                    clinical_data[patient_id]["Number of metastases"] = prev_met + new_met
                else:
                    # Adicionar um novo paciente ao dicionário
                    clinical_data[patient_id] = extracted_data
                    print(f"This patient is new!")


  #CSV without dups
    for patient_id, data in clinical_data.items():
        writer.writerow({
            'Patient ID': patient_id,
            'Sex': data['Sex'],
            'Number of metastases': data['Number of metastases'],
            'Machine': data['Machine']
        })

    print(f"File '{csv_file}' updated successfully!")




Extracting patient's data...
Patient ID: Patient_61
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_61', 'Sex': 'F', 'Number of metastases': 2, 'Machine': 'LINAC'}
Processing patient: Patient_61
Patient Patient_61 already exists.

Extracting patient's data...
Patient ID: Patient_78
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_78', 'Sex': 'F', 'Number of metastases': 22, 'Machine': 'Gamma Knife'}
Processing patient: Patient_78
Patient Patient_78 already exists.

Extracting patient's data...
Patient ID: Patient_94
Patient Sex: M
Data extraction completed with success {'Patient ID': 'Patient_94', 'Sex': 'M', 'Number of metastases': 3, 'Machine': 'Gamma Knife'}
Processing patient: Patient_94
Patient Patient_94 already exists.

Extracting patient's data...
Patient ID: Patient_53
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_53', 'Sex': 'F', 'Number of metastases': 2, 'Machine': 'LINAC'}
Processin

Insert please the type of machine used - LINAC or Gamma Knife LINAC


Data extraction completed with success {'Patient ID': 'Patient_2-25', 'Sex': 'F', 'Number of metastases': 4, 'Machine': 'LINAC'}
Processing patient: Patient_2-25
Patient Patient_2-25 already exists.

Extracting patient's data...
Patient ID: Patient_75
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_75', 'Sex': 'F', 'Number of metastases': 2, 'Machine': 'Gamma Knife'}
Processing patient: Patient_75
Patient Patient_75 already exists.

Extracting patient's data...
Patient ID: Patient_84
Patient Sex: M
Data extraction completed with success {'Patient ID': 'Patient_84', 'Sex': 'M', 'Number of metastases': 2, 'Machine': 'LINAC'}
Processing patient: Patient_84
Patient Patient_84 already exists.

Extracting patient's data...
Patient ID: Patient_88
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_88', 'Sex': 'F', 'Number of metastases': 2, 'Machine': 'LINAC'}
Processing patient: Patient_88
Patient Patient_88 already exists.

Extracting 

Insert please the type of machine used - LINAC or Gamma Knife LINAC


Data extraction completed with success {'Patient ID': 'Patient_1-25', 'Sex': 'F', 'Number of metastases': 10, 'Machine': 'LINAC'}
Processing patient: Patient_1-25
Patient Patient_1-25 already exists.

Extracting patient's data...
Patient ID: Patient_68
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_68', 'Sex': 'F', 'Number of metastases': 2, 'Machine': 'LINAC'}
Processing patient: Patient_68
Patient Patient_68 already exists.

Extracting patient's data...
Patient ID: Patient_68
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_68', 'Sex': 'F', 'Number of metastases': 2, 'Machine': 'LINAC'}
Processing patient: Patient_68
Patient Patient_68 already exists.

Extracting patient's data...
Patient ID: Patient_63
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_63', 'Sex': 'F', 'Number of metastases': 2, 'Machine': 'LINAC'}
Processing patient: Patient_63
Patient Patient_63 already exists.

Extracting patie

Insert please the type of machine used - LINAC or Gamma Knife LINAC


Data extraction completed with success {'Patient ID': 'Patient_3-25', 'Sex': 'F', 'Number of metastases': 10, 'Machine': 'LINAC'}
Processing patient: Patient_3-25
Patient Patient_3-25 already exists.

Extracting patient's data...
Patient ID: Patient_81
Patient Sex: M
Data extraction completed with success {'Patient ID': 'Patient_81', 'Sex': 'M', 'Number of metastases': 2, 'Machine': 'LINAC'}
Processing patient: Patient_81
Patient Patient_81 already exists.

Extracting patient's data...
Patient ID: Patient_83
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_83', 'Sex': 'F', 'Number of metastases': 2, 'Machine': 'LINAC'}
Processing patient: Patient_83
Patient Patient_83 already exists.

Extracting patient's data...
Patient ID: Patient_91
Patient Sex: F
Data extraction completed with success {'Patient ID': 'Patient_91', 'Sex': 'F', 'Number of metastases': 6, 'Machine': 'Gamma Knife'}
Processing patient: Patient_91
Patient Patient_91 already exists.

Extracting

In [20]:
df = pd.read_csv("/notebooks/disk2/CSV_Dataset/Clinical_Features.csv")

mapa_ids = {
    "Patient_1-25": "Patient_55",
    "Patient_2-25": "Patient_56",
    "Patient_3-25": "Patient_57"
}
df["Patient ID"] = df["Patient ID"].replace(mapa_ids)

# Guardar ficheiro alterado
df.to_csv("/notebooks/disk2/CSV_Dataset/Clinical_Features.csv", index=False)


In [26]:
df = pd.read_csv("/notebooks/disk2/CSV_Dataset/Clinical_Features.csv")

df = df.sort_values(by="Patient ID", ascending=True)

df.to_csv("/notebooks/disk2/CSV_Dataset/Clinical_Features.csv", index=False)

### Clinical Features from information given

In [ ]:
import pandas as pd
df = pd.read_excel("/notebooks/disk2/CSV_Dataset/D-Cohort-Brain.xlsx", header=2, usecols="B:F")
df.rename(columns={"Primary ID": "Patient ID"}, inplace=True)
# Save to CSV, keeping same columns
df.to_csv("/notebooks/disk2/CSV_Dataset/D-Cohort-Brain.csv", index=False, encoding="utf-8")

print("Columns saved:", df.columns.tolist())

In [ ]:
df = pd.read_csv('/notebooks/disk2/CSV_Dataset/Clinical_Features.csv')
ds = pd.read_csv ('/notebooks/disk2/CSV_Dataset/D-Cohort-Brain.csv')
common_column = "Patient ID"
df_1 = set(df[common_column])
ds_1 = set(ds[common_column])

df_ds = df_1 & ds_1
df_combined = pd.merge(df, ds, on=common_column, how="inner")
df_combined.to_csv('/notebooks/disk2/CSV_Dataset/Extra_Distribution_Clinical_Features.csv', index=False)

In [33]:
df = pd.read_csv("/notebooks/disk2/CSV_Dataset/Extra_Distribution_Clinical_Features.csv")

df["Decision"] = None

for i, pid in enumerate(df["Patient ID"]):
    value = input(f"Enter the decision for Patient {pid}: ")
    df.at[i, "Decision"] = value

df.to_csv("/notebooks/disk2/CSV_Dataset/Clinical_Features_Final.csv", index=False)

print("All values entered and file saved as clinical_features_final.csv")

Enter the decision for Patient Patient_01:  2nd Treatment
Enter the decision for Patient Patient_07:  2nd Treatment
Enter the decision for Patient Patient_08:  2nd Treatment
Enter the decision for Patient Patient_09:  2nd Treatment
Enter the decision for Patient Patient_10:  2nd Treatment
Enter the decision for Patient Patient_11:  2nd Treatment
Enter the decision for Patient Patient_12:  2nd Treatment
Enter the decision for Patient Patient_14:  2nd Treatment
Enter the decision for Patient Patient_17:  2nd Treatment
Enter the decision for Patient Patient_19:  2nd Treatment
Enter the decision for Patient Patient_25:  2nd Treatment
Enter the decision for Patient Patient_26:  2nd Treatment
Enter the decision for Patient Patient_27:  2nd Treatment
Enter the decision for Patient Patient_28:  2nd Treatment
Enter the decision for Patient Patient_29:  2nd Treatment
Enter the decision for Patient Patient_33:  2nd Treatment
Enter the decision for Patient Patient_34:  2nd Treatment
Enter the deci

All values entered and file saved as clinical_features_final.csv


### Filter the patients that we need to exclude from the reasons mentioned in the thesis

In [3]:
df = pd.read_csv("/notebooks/disk2/CSV_Dataset/Clinical_Features_Final.csv")
exclude_ids = [
    "Patient_02", "Patient_03", "Patient_04", "Patient_05", "Patient_06",
    "Patient_13", "Patient_15", "Patient_16", "Patient_18", "Patient_20",
    "Patient_21", "Patient_22", "Patient_23", "Patient_24", "Patient_30",
    "Patient_31", "Patient_32", "Patient_36", "Patient_39", "Patient_40",
    "Patient_41", "Patient_42", "Patient_48", "Patient_50", "Patient_51",
    "Patient_52", "Patient_53", "Patient_55", "Patient_56", "Patient_57",
    "Patient_58","Patient_59", "Patient_60", "Patient_61", "Patient_64", 
    "Patient_66","Patient_70", "Patient_79", "Patient_80", "Patient_81",
    "Patient_86", "Patient_92", "Patient_93", "Patient_97"
]

df_filtered = df[~df["Patient ID"].isin(exclude_ids)]

df_filtered.to_csv("/notebooks/disk2/CSV_Dataset/Clinical_Features_Final_Final.csv", index=False)